# Ingestão Bronze - Cadastro de Cliente

Este notebook realiza a leitura de um arquivo Excel contendo o cadastro de clientes
e grava os dados em formato Delta na camada **Bronze**.

**Origem dos dados:**
- Arquivo: `D:\Projetos\Jornada_financas_pessoais\data\source\cadastro_cliente.xlsx`

**Destino:**
- Caminho Delta: `D:\Projetos\Jornada_financas_pessoais\data\delta\bronze\raw_cadcliente`
- Tabela Delta: `bronze.raw_cadcliente`


## Imports

In [8]:
from delta import configure_spark_with_delta_pip
from pyspark.sql import SparkSession
import pandas as pd

## Start Spark Session

In [9]:
# Inicializa uma SparkSession
builder = SparkSession.builder \
    .appName("Leitura de Arquivo de Formato Fixo") \
    .master("local[*]") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

print(f"\n✅ Spark {spark.version} iniciado!\n")


✅ Spark 3.5.7 iniciado!



## Variables

In [10]:
# Define o caminho do diretório dos arquivos de origem
source_path = "D:/Projetos/Jornada_financas_pessoais/data/source"

# Define caminhos locais onde serão armazenadas as tabelas Delta
base_bronze_path = "D:/Projetos/Jornada_financas_pessoais/data/delta/bronze"

# Define o caminho da tabela Delta Source
delta_path_cadcliente = f"{base_bronze_path}/raw_cadcliente"

## Leitura do arquivo Excel

In [ ]:
# Lê o Excel com pandas
df_cadcliente_pd = pd.read_excel(
    f'{source_path}/Cadastro_Cliente.xlsx',
    engine='openpyxl',
    dtype=str  # Força todas as colunas como string
)

# Renomeia a coluna DATA NASCIMENTO
df_cadcliente_pd.rename(columns={'DATA NASCIMENTO': 'data_nascimento'}, inplace=True)

# Converte para PySpark DataFrame
df_cadcliente = spark.createDataFrame(df_cadcliente_pd)

root
 |-- CPF: string (nullable = true)
 |-- NOME: string (nullable = true)
 |-- data_nascimento: string (nullable = true)
 |-- EMAIL: string (nullable = true)
 |-- TELEFONE: string (nullable = true)

+-----------+--------------------+-------------------+--------------------+-----------+
|        CPF|                NOME|    data_nascimento|               EMAIL|   TELEFONE|
+-----------+--------------------+-------------------+--------------------+-----------+
|11272738841|FLÁVIO CAVALCANTE...|1970-05-11 00:00:00|fuyemura@hotmail.com|11992995588|
|19114900874|PATRÍCIA REGINA D...|1975-12-16 00:00:00|patriciaoliveira1...|11992995588|
+-----------+--------------------+-------------------+--------------------+-----------+



## Save Data Frame

In [15]:
# Grava em formato Delta
(
    df_cadcliente.write.format("delta")
    .mode("overwrite")
    .option("overwriteSchema", "false")
    .save(delta_path_cadcliente)
)

print(f"[SUCESSO] DataFrame gravado no Delta Lake em: {delta_path_cadcliente}")

[SUCESSO] DataFrame gravado no Delta Lake em: D:/Projetos/Jornada_financas_pessoais/data/delta/bronze/raw_cadcliente


## Stop Spark Session

In [16]:
# Encerra a SparkSession
spark.stop()